# Telegram: 1. Basic

The following tutorial shows how to run a regular Chatsky script in Telegram.
It asks users for the '/start' command and then loops in one place.

Here, [LongpollingInterface](../apiref/chatsky.messengers.telegram.interface.rst#chatsky.messengers.telegram.interface.LongpollingInterface)
class and [python-telegram-bot](https://docs.python-telegram-bot.org/)
library are used for accessing telegram API in polling mode.

Telegram API token is required to access telegram API.

In [1]:
# installing dependencies
%pip install -q chatsky[telegram]


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

from chatsky.script import conditions as cnd
from chatsky.script import labels as lbl
from chatsky.script import RESPONSE, TRANSITIONS, Message
from chatsky.messengers.telegram import LongpollingInterface
from chatsky.pipeline import Pipeline
from chatsky.utils.testing.common import is_interactive_mode

In order to integrate your script with Telegram, you need an instance of the
[_AbstractTelegramInterface](../apiref/chatsky.messengers.telegram.abstract.rst#chatsky.messengers.telegram.abstract._AbstractTelegramInterface) class.
Two of its child subclasses that can be instantiated
are [LongpollingInterface](../apiref/chatsky.messengers.telegram.interface.rst#chatsky.messengers.telegram.interface.LongpollingInterface) and
[WebhookInterface](../apiref/chatsky.messengers.telegram.interface.rst#chatsky.messengers.telegram.interface.WebhookInterface).
The latter requires a webserver, so here we use long polling interface.

[_AbstractTelegramInterface](../apiref/chatsky.messengers.telegram.abstract.rst#chatsky.messengers.telegram.abstract._AbstractTelegramInterface)
encapsulates the bot logic. The only required
argument for a bot to run is a token. Some other parameters
(such as host, port, interval, etc.) can be passed as keyword arguments
(for their specs see [documentations of the child subclasses](
../apiref/chatsky.messengers.telegram.interface.rst
).

Either of the two interfaces connect the bot to Telegram.
They can be passed directly to a Chatsky `Pipeline` instance.

In [3]:
script = {
    "greeting_flow": {
        "start_node": {
            TRANSITIONS: {"greeting_node": cnd.exact_match("/start")},
        },
        "greeting_node": {
            RESPONSE: Message("Hi"),
            TRANSITIONS: {lbl.repeat(): cnd.true()},
        },
        "fallback_node": {
            RESPONSE: Message("Please, repeat the request"),
            TRANSITIONS: {"greeting_node": cnd.exact_match("/start")},
        },
    }
}

In [4]:
interface = LongpollingInterface(token=os.environ["TG_BOT_TOKEN"])

In [5]:
pipeline = Pipeline.from_script(
    script=script,
    start_label=("greeting_flow", "start_node"),
    fallback_label=("greeting_flow", "fallback_node"),
    messenger_interface=interface,
    # The interface can be passed as a pipeline argument.
)


if __name__ == "__main__":
    if is_interactive_mode():
        # prevent run during doc building
        pipeline.run()